In [1]:
import numpy as np
# requires: pip install anytree
from anytree import Node, RenderTree

In [2]:
def bitboard():
    # defines basic bitboards of type boolean
    return np.zeros((8,8), dtype=bool)

def give_bitboards():
    # creates bitboards
    bb = {
        # piece colors
        "W" : bitboard(),
        "B" : bitboard(),
        # pieces
        "k" : bitboard(),
        "q" : bitboard(),
        "n" : bitboard(),
        "r" : bitboard(),
        "b" : bitboard(),
        "p" : bitboard(),
    }
    return bb


def give_static_bitboards():
    # creates static bitboards
    sbb = {
        # lines
        "la" : bitboard(),
        "lb" : bitboard(),
        "lc" : bitboard(),
        "ld" : bitboard(),
        "le" : bitboard(),
        "lf" : bitboard(),
        "lg" : bitboard(),
        "lh" : bitboard(),
        # rows
        "1" : bitboard(),
        "2" : bitboard(),
        "3" : bitboard(),
        "4" : bitboard(),
        "5" : bitboard(),
        "6" : bitboard(),
        "7" : bitboard(),
        "8" : bitboard(),
    }
    return sbb

sbb = give_static_bitboards()

        


def init_game(b):
    # initializes board with pieces
    
    # colored pieces for black/white player
    b['W'][0:2,:] = True
    b['B'][-2:,:] = True
    
    # adds kings('k'), queens, knights, rooks, bishops and pawns to the board on respecting fields
    b['k'][[0,-1],[4,4]] = True
    b['q'][[0,-1],[3,3]] = True
    b['n'][[0,0,-1,-1],[1,-2,1,-2]] = True
    b['r'][[0,0,-1,-1],[0,-1,0,-1]] = True
    b['b'][[0,0,-1,-1],[2,-3,2,-3]] = True
    b['p'][[1,-2],:] = True
    
    # lines
    sbb['la'][:,0] = True
    sbb['lb'][:,1] = True
    sbb['lc'][:,2] = True
    sbb['ld'][:,3] = True
    sbb['le'][:,4] = True
    sbb['lf'][:,5] = True
    sbb['lg'][:,6] = True
    sbb['lh'][:,7] = True
    
    # rows
    sbb['1'][0,:] = True
    sbb['2'][1,:] = True
    sbb['3'][2,:] = True
    sbb['4'][3,:] = True
    sbb['5'][4,:] = True
    sbb['6'][5,:] = True
    sbb['7'][6,:] = True
    sbb['8'][7,:] = True
    
    return b


def print_board(b, flip=False):
    # pretty prints board
    # flip: white at bottom
    board = np.empty((8,8), dtype=str)
    board[:] = '_'
    
    # black player: lower case, white player: UPPER CASE
    board[b['B'] & b['k']] = 'k'
    board[b['B'] & b['q']] = 'q'
    board[b['B'] & b['n']] = 'n'
    board[b['B'] & b['r']] = 'r'
    board[b['B'] & b['b']] = 'b'
    board[b['B'] & b['p']] = 'p'
    
    board[b['W'] & b['k']] = 'K'
    board[b['W'] & b['q']] = 'Q'
    board[b['W'] & b['n']] = 'N'
    board[b['W'] & b['r']] = 'R'
    board[b['W'] & b['b']] = 'B'
    board[b['W'] & b['p']] = 'P'

    cols = np.reshape([['a','b','c','d','e','f','g','h'],['-','-','-','-','-','-','-','-']],(2,8))
    rows = np.reshape([['/','-','1','2','3','4','5','6','7','8'],['|','+','|','|','|','|','|','|','|','|']],(2,10))
    
    board = np.concatenate((cols, board), axis=0)
    board = np.concatenate((rows.T, board), axis=1)
    
    if flip == True:
        board = np.flip(board, axis=0)
    return board

In [3]:

### QUIET MOVES ###

def moves_quiet_pawns_W(b):
    # shows possible normal moves of Black pawns
    return np.roll((b['W'] & b['p']), 8) & ~(b['W'] | b['B'])

def moves_quiet_pawns_B(b):
    # shows possible normal moves of Black pawns
    
    #((B & SCH) >> 8) & ~(WEI | SCH)
    return np.roll((b['B'] & b['p']), -8) & ~(b['W'] | b['B'])


def moves_quiet_king_W(b):
    # king can only move up if it is not in the upper row, etc. 
    up    = np.roll((b['W'] & b['k'] & ~sbb['8']),   8) & ~(b['W'] | b['B'])
    down  = np.roll((b['W'] & b['k'] & ~sbb['1']),  -8) & ~(b['W'] | b['B'])
    left  = np.roll((b['W'] & b['k'] & ~sbb['la']), -1) & ~(b['W'] | b['B'])
    right = np.roll((b['W'] & b['k'] & ~sbb['lh']),  1) & ~(b['W'] | b['B'])
    return (up|down|left|right)


def moves_quiet_king_B(b):
    # king can only move up if it is not in the upper row, etc. 
    up    = np.roll((b['B'] & b['k'] & ~sbb['8']),   8) & ~(b['W'] | b['B'])
    down  = np.roll((b['B'] & b['k'] & ~sbb['1']),  -8) & ~(b['W'] | b['B'])
    left  = np.roll((b['B'] & b['k'] & ~sbb['la']), -1) & ~(b['W'] | b['B'])
    right = np.roll((b['B'] & b['k'] & ~sbb['lh']),  1) & ~(b['W'] | b['B'])
    return (up|down|left|right)

### ATTACK MOVES ###

def moves_attack_king_W(b):
    # king can only move up if it is not in the upper row, etc. 
    up    = np.roll((b['W'] & b['k'] & ~sbb['8']),   8) & ~(b['W']) & (b['B'])
    down  = np.roll((b['W'] & b['k'] & ~sbb['1']),  -8) & ~(b['W']) & (b['B'])
    left  = np.roll((b['W'] & b['k'] & ~sbb['la']), -1) & ~(b['W']) & (b['B'])
    right = np.roll((b['W'] & b['k'] & ~sbb['lh']),  1) & ~(b['W']) & (b['B'])
    return (up|down|left|right)

def moves_attack_king_B(b):
    # king can only move up if it is not in the upper row, etc. 
    up    = np.roll((b['B'] & b['k'] & ~sbb['8']),   8) & (b['W']) & ~(b['B'])
    down  = np.roll((b['B'] & b['k'] & ~sbb['1']),  -8) & (b['W']) & ~(b['B'])
    left  = np.roll((b['B'] & b['k'] & ~sbb['la']), -1) & (b['W']) & ~(b['B'])
    right = np.roll((b['B'] & b['k'] & ~sbb['lh']),  1) & (b['W']) & ~(b['B'])
    return (up|down|left|right)



In [19]:
### ZUG-SIMULATOR ###



#ziehende Figur: BB(From) OR BB(To) XOR BB(gezogene Figur)
#geschlagene Figur: bb(geschlagene Figur) XOR BB(To) -> (Figur)
'''
def make_move_quiet(b, bb_from, bb_to, bb_piece):
    for bb in b:
        b[bb] = (bb_from | bb_to) ^ bb_piece
    return b

def make_move_attack(b, bb_from, bb_to, bb_piece, bb_enemy_piece):
    for bb in b:
        b[bb] = (bb_from | bb_to) ^ bb_piece
        b[bb] = (bb_enemy_piece ^ bb_to) ^ bb_piece
    return b
'''
# eigene Version
'''

def make_move_quiet(b, bb_from, bb_to):
    for bb in b:
        if (bb & bb_from).any() : # falls eigene figur auf bitboard verhanden
            bb = (bb | bb_to) ^ bb_from # update bitboard : Feld auf to entsteht, Feld auf from verschwindet
    return b

def make_move_attack(b, bb_from, bb_to):
    for bb in b: # Schlage gegnerische Figur:
        if (bb & bb_to).any() : # falls gegnerische figur auf ziel auf bitboard verhanden
            bb = bb  ^ bb_to # update bitboard: Feld auf to verschwindet
            
    b = make_move_quiet(b, bb_from, bb_to) # Führe anschließend "stillen" Zug aus -> Optimierung: eine Schleife
    return b
'''

def make_move(b, bb_from, bb_to):
    for bb in b:
        if (bb & bb_to).any() : # falls gegnerische figur auf ziel auf bitboard verhanden
            bb = bb  ^ bb_to # update bitboard: Schlag: Feld auf bb_to verschwindet
        if (bb & bb_from).any() : # falls eigene figur auf bitboard verhanden
            bb = (bb | bb_to) ^ bb_from # update bitboard : Feld auf bb_to entsteht, Feld auf bb_from verschwindet
    return b




In [10]:
### ZUGGENERATOR ###

### https://www.chessprogramming.org/Pieces_versus_Directions

'''

for all pieces {
  generateMovetargets(piece);
  for all capture targets
      generate captures (move or capture list)
  for all empty square targets
      generate quite moves (move list)
}

'''

def generate_moves(b, player):
    # generate a searchtree and search for possible pseudolegal moves
    moves = Node('root')
    # add possible striking moves
    
    # add plain movement moves
    
    return moves




In [6]:
b = init_game(give_bitboards())
sbb = give_static_bitboards()
print(print_board(b))

#moves_quiet_king_W(b)

[['/' '|' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h']
 ['-' '+' '-' '-' '-' '-' '-' '-' '-' '-']
 ['1' '|' 'R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']
 ['2' '|' 'P' 'P' 'P' 'P' 'P' 'P' 'P' 'P']
 ['3' '|' '_' '_' '_' '_' '_' '_' '_' '_']
 ['4' '|' '_' '_' '_' '_' '_' '_' '_' '_']
 ['5' '|' '_' '_' '_' '_' '_' '_' '_' '_']
 ['6' '|' '_' '_' '_' '_' '_' '_' '_' '_']
 ['7' '|' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['8' '|' 'r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']]


In [7]:
player = 1
generate_moves(b,player)

Node('/root')

In [18]:
if (b['k'] & b['W']).any():
    print('ok')

ok
